In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# **Create Delta Table**

## using sql api

In [0]:
%sql
create table delta.deltaschema.deltatable
(
  id int not null,
  name string,
  age int
)
using delta

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5945745070037240>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'create table delta.deltaschema.deltatable\n(\n  id int not null,\n  name string,\n  age int\n)\nusing delta\n')

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:147, in SqlMagic.sql(self, line, cell)
    1

## insert values

In [0]:
%sql
insert into delta.deltaschema.deltatable(id,name,age) values
(1,"suresh",25),
(2,"kumar",30),
(3,"ram",35),
(4,"murugan",40)

num_affected_rows,num_inserted_rows
4,4


## read the data

In [0]:
%sql
select * from delta.deltaschema.deltatable

## create dataframe

In [0]:
data = [(1, 'John',25),(2, 'Smith',30),(3, 'Sarah',35),(4, 'Mary',40),(5, 'Mike',45)]


schema = ['id', 'name', 'age']

df = spark.createDataFrame(data,schema)

display(df)

id,name,age
1,John,25
2,Smith,30
3,Sarah,35
4,Mary,40
5,Mike,45


## write data

In [0]:
df.write.format("delta")\
    .mode("append")\
        .save("/Volumes/delta/deltaschema/sink")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5945745070037255>, line 3
      1 df.write.format("delta")\
      2     .mode("append")\
----> 3         .save("/Volumes/delta/deltaschema/sink")

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/readwriter.py:679, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
    677     self.format(format)
    678 self._write.path = path
--> 679 _, _, ei = self._spark.client.execute_command(
    680     self._write.command(self._spark.client), self._write.observations
    681 )
    682 self._callback(ei)

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1297, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1295     req.user_context.user_id = self._user_id
   1296 req.plan.command.CopyFrom(command)
-> 12

## read delta data that means parquet file

In [0]:
%sql
select * from delta.`/Volumes/delta/deltaschema/sink`

cust_id,cust_name,cust_age,cust_city
1,John,25,chennai
2,Smith,30,salem
3,Sarah,35,covai
4,Mary,40,trichy
5,Mike,45,ooty


# **SCHEMA ENFORCEMENT**

In [0]:
data1 = [(1, 'John',25,"chennai"),(2, 'Smith',30,"salem"),(3, 'Sarah',35,"covai"),(4, 'Mary',40,"trichy"),(5, 'Mike',45,"ooty")]


schema1 = ['id','name','age','city']

df1 = spark.createDataFrame(data1,schema1)

display(df1)

## here i already append one data frame in /Volumes/delta/deltaschema/sink this location schema is ['id', 'name', 'age']  but now i add other dataframe with different schema ['id','name','age','city']  can't show error we use SCHEMA EVOLUTION

In [0]:
df1.write.format("delta")\
  .mode("append")\
    .save("/Volumes/delta/deltaschema/sink")

# **SCHEMA EVOLUTION**

In [0]:
df1.write.format("delta")\
    .mode("append")\
    .option("mergeSchema","true")\
    .save("/Volumes/delta/deltaschema/sink")

In [0]:
%sql
select * from delta.`/Volumes/delta/deltaschema/sink`

# **SCHEMA OVERWRITE**

## here now i change schema name above add new schema in dataframe and write same location

In [0]:
data2 = [(1, 'John',25,"chennai"),(2, 'Smith',30,"salem"),(3, 'Sarah',35,"covai"),(4, 'Mary',40,"trichy"),(5, 'Mike',45,"ooty")]


schema2 = ['cust_id','cust_name','cust_age','cust_city']

df2 = spark.createDataFrame(data2,schema2)

display(df2)

In [0]:
df2.write.format("delta")\
    .mode("append")\
    .save("/Volumes/delta/deltaschema/sink")

In [0]:
df2.write.format("delta")\
    .mode("overwrite")\
    .save("/Volumes/delta/deltaschema/sink")

In [0]:
df2.write.format("delta")\
    .mode("overwrite")\
    .option("overwriteSchema","true")\
    .save("/Volumes/delta/deltaschema/sink")

In [0]:
%sql
select * from delta.`/Volumes/delta/deltaschema/sink`

cust_id,cust_name,cust_age,cust_city
1,John,25,chennai
2,Smith,30,salem
3,Sarah,35,covai
4,Mary,40,trichy
5,Mike,45,ooty


# **DML**

### **update**

In [0]:
%sql
update delta.`/Volumes/delta/deltaschema/sink`
set cust_name = "john cena"
where cust_id = 1
    


num_affected_rows
1


In [0]:
%sql
select * from delta.`/Volumes/delta/deltaschema/sink`


cust_id,cust_name,cust_age,cust_city
1,john cena,25,chennai
2,Smith,30,salem
3,Sarah,35,covai
4,Mary,40,trichy
5,Mike,45,ooty


# **UPSERT**

In [0]:
data3 = [(6, 'aaaaa',25,"chennai"),(2, 'Smithaaaaaa',30,"salem"),(7, 'bbbb',35,"covai"),(8, 'cccc',40,"trichy"),(5, 'Mikeeeeeeee',45,"ooty")]

schema3 = ['cust_id','cust_name','cust_age','cust_city']

df3 = spark.createDataFrame(data3,schema3)

display(df3)

cust_id,cust_name,cust_age,cust_city
6,aaaaa,25,chennai
2,Smithaaaaaa,30,salem
7,bbbb,35,covai
8,cccc,40,trichy
5,Mikeeeeeeee,45,ooty


In [0]:
from delta.tables import DeltaTable

In [0]:
dlt_obj = DeltaTable.forPath(spark,"/Volumes/delta/deltaschema/sink")


dlt_obj.alias("target").merge(df3.alias("source"), "target.cust_id = source.cust_id")\
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from delta.`/Volumes/delta/deltaschema/sink`

cust_id,cust_name,cust_age,cust_city
1,john cena,25,chennai
3,Sarah,35,covai
4,Mary,40,trichy
6,aaaaa,25,chennai
2,Smithaaaaaa,30,salem
7,bbbb,35,covai
8,cccc,40,trichy
5,Mikeeeeeeee,45,ooty


# **TABLE UTILITY COMMANDS**

In [0]:
%sql
describe delta.deltaschema.deltatable

col_name,data_type,comment
id,int,null
name,string,null
age,int,null


## volumes

In [0]:
%sql
describe delta.`/Volumes/delta/deltaschema/sink`

col_name,data_type,comment
cust_id,bigint,null
cust_name,string,null
cust_age,bigint,null
cust_city,string,null


In [0]:
%sql
describe extended delta.deltaschema.deltatable

col_name,data_type,comment
id,int,null
name,string,null
age,int,null
,,
# Delta Statistics Columns,,
Column Names,"id, name, age",
Column Selection Method,first-32,
,,
# Detailed Table Information,,
Catalog,delta,


## volumes

In [0]:
%sql
describe extended delta.`/Volumes/delta/deltaschema/sink`

col_name,data_type,comment
cust_id,bigint,null
cust_name,string,null
cust_age,bigint,null
cust_city,string,null
,,
# Delta Statistics Columns,,
Column Names,"cust_id, cust_name, cust_age, cust_city",
Column Selection Method,first-32,
,,
# Detailed Table Information,,


In [0]:
%sql
describe history delta.deltaschema.deltatable

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-07-12T07:06:26.000Z,5763104327656046,dshkr2002@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> true, partitionBy -> [])",null,null,0712-055808-xwnwn6c7-v2n,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 4, numOutputBytes -> 1017)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
1,2025-07-12T06:00:51.000Z,5763104327656046,dshkr2002@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> true, partitionBy -> [])",null,null,0712-055808-xwnwn6c7-v2n,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 4, numOutputBytes -> 1017)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
0,2025-07-12T05:58:29.000Z,5763104327656046,dshkr2002@gmail.com,CREATE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)",null,null,0712-055808-xwnwn6c7-v2n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12


In [0]:
%sql
describe history delta.`/Volumes/delta/deltaschema/sink`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
7,2025-07-12T07:22:20.000Z,5763104327656046,dshkr2002@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,null,0712-055808-xwnwn6c7-v2n,6,SnapshotIsolation,false,"Map(numRemovedFiles -> 6, numRemovedBytes -> 7768, p25FileSize -> 1512, numDeletionVectorsRemoved -> 1, minFileSize -> 1512, numAddedFiles -> 1, maxFileSize -> 1512, p75FileSize -> 1512, p50FileSize -> 1512, numAddedBytes -> 1512)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
6,2025-07-12T07:22:18.000Z,5763104327656046,dshkr2002@gmail.com,MERGE,"Map(predicate -> [""(cust_id#13431L = cust_id#13398L)""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,null,0712-055808-xwnwn6c7-v2n,5,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 5, numTargetBytesAdded -> 6368, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 1, numTargetRowsMatchedUpdated -> 2, executionTimeMs -> 2890, materializeSourceTimeMs -> 154, numTargetRowsInserted -> 3, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 1091, numTargetRowsUpdated -> 2, numOutputRows -> 5, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 5, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 1558)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
5,2025-07-12T07:12:13.000Z,5763104327656046,dshkr2002@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,null,0712-055808-xwnwn6c7-v2n,4,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 2680, p25FileSize -> 1400, numDeletionVectorsRemoved -> 1, minFileSize -> 1400, numAddedFiles -> 1, maxFileSize -> 1400, p75FileSize -> 1400, p50FileSize -> 1400, numAddedBytes -> 1400)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
4,2025-07-12T07:12:11.000Z,5763104327656046,dshkr2002@gmail.com,UPDATE,"Map(predicate -> [""(cust_id#12571L = 1)""])",null,null,0712-055808-xwnwn6c7-v2n,3,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2831, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1710, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1294, rewriteTimeMs -> 1102)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
3,2025-07-12T07:06:47.000Z,5763104327656046,dshkr2002@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,null,0712-055808-xwnwn6c7-v2n,2,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 1, numRemovedBytes -> 1386, numOutputRows -> 5, numOutputBytes -> 1386)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
2,2025-07-12T07:02:25.000Z,5763104327656046,dshkr2002@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,null,0712-055808-xwnwn6c7-v2n,1,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 2, numRemovedBytes -> 2421, numOutputRows -> 5, numOutputBytes -> 1386)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
1,2025-07-12T06:44:54.000Z,5763104327656046,dshkr2002@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,null,0712-055808-xwnwn6c7-v2n,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 5, numOutputBytes -> 1326)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
0,2025-07-12T06:25:02.000Z,5763104327656046,dshkr2002@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partiti

In [0]:
%sql
select * from delta.`/Volumes/delta/deltaschema/sink`
version as of 4

cust_id,cust_name,cust_age,cust_city
2,Smith,30,salem
3,Sarah,35,covai
4,Mary,40,trichy
5,Mike,45,ooty
1,john cena,25,chennai


In [0]:
%sql
select * from delta.`/Volumes/delta/deltaschema/sink`

cust_id,cust_name,cust_age,cust_city
1,john cena,25,chennai
3,Sarah,35,covai
4,Mary,40,trichy
6,aaaaa,25,chennai
2,Smithaaaaaa,30,salem
7,bbbb,35,covai
8,cccc,40,trichy
5,Mikeeeeeeee,45,ooty


In [0]:
%sql
restore delta.`/Volumes/delta/deltaschema/sink`
version as of 4

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
2680,2,1,2,1512,2680


In [0]:
%sql
select * from delta.`/Volumes/delta/deltaschema/sink`

cust_id,cust_name,cust_age,cust_city
2,Smith,30,salem
3,Sarah,35,covai
4,Mary,40,trichy
5,Mike,45,ooty
1,john cena,25,chennai


In [0]:
%sql
show tblproperties delta.deltaschema.deltatable

key,value
delta.enableDeletionVectors,true
delta.feature.appendOnly,supported
delta.feature.deletionVectors,supported
delta.feature.invariants,supported
delta.minReaderVersion,3
delta.minWriterVersion,7


In [0]:
%sql
create table delta.deltaschema.deltatable2
clone delta.deltaschema.deltatable
    


source_table_size,source_num_of_files,num_of_synced_transactions,num_removed_files,num_copied_files,removed_files_size,copied_files_size
2034,2,null,0,2,0,2034


In [0]:
data4 = [(1,"aaa","chennai"),(2,"bbb","salem"),(3,"ccc","covai"),(4,"ddd","trichy")]


schema4 = ['id','name','city']

df4 = spark.createDataFrame(data4,schema4)

display(df4)

id,name,city
1,aaa,chennai
2,bbb,salem
3,ccc,covai
4,ddd,trichy


In [0]:
df4.write.format("delta")\
    .mode("append")\
    .save("/Volumes/delta/deltaschema/optimization/optimize")


In [0]:
%sql
describe history delta.`/Volumes/delta/deltaschema/optimization/optimize/`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2025-07-12T13:49:06.000Z,5763104327656046,dshkr2002@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,null,0712-134530-qhpwm4jy-v2n,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 4, numOutputBytes -> 1076)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12


In [0]:
%sql
update delta.`/Volumes/delta/deltaschema/optimization/optimize/`
set name = 'updated'
where id = 1

num_affected_rows
1


In [0]:
%sql
describe history delta.`/Volumes/delta/deltaschema/optimization/optimize/`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-07-12T13:51:30.000Z,5763104327656046,dshkr2002@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,null,0712-134530-qhpwm4jy-v2n,1,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 2106, p25FileSize -> 1089, numDeletionVectorsRemoved -> 1, minFileSize -> 1089, numAddedFiles -> 1, maxFileSize -> 1089, p75FileSize -> 1089, p50FileSize -> 1089, numAddedBytes -> 1089)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
1,2025-07-12T13:51:28.000Z,5763104327656046,dshkr2002@gmail.com,UPDATE,"Map(predicate -> [""(id#10828L = 1)""])",null,null,0712-134530-qhpwm4jy-v2n,0,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3493, numDeletionVectorsUpdated -> 0, scanTimeMs -> 2106, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1030, rewriteTimeMs -> 1352)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
0,2025-07-12T13:49:06.000Z,5763104327656046,dshkr2002@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,null,0712-134530-qhpwm4jy-v2n,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 4, numOutputBytes -> 1076)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12


## optimize

In [0]:
%sql
optimize delta.`/Volumes/delta/deltaschema/optimization/optimize/`

path,metrics
dbfs:/Volumes/delta/deltaschema/optimization/optimize,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1752328384737, 1752328385189, 8, 0, null, List(0, 0), null, 3, 3, 0, 0, null)"


In [0]:
%sql
describe history delta.`/Volumes/delta/deltaschema/optimization/optimize/`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-07-12T13:51:30.000Z,5763104327656046,dshkr2002@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,null,0712-134530-qhpwm4jy-v2n,1,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 2106, p25FileSize -> 1089, numDeletionVectorsRemoved -> 1, minFileSize -> 1089, numAddedFiles -> 1, maxFileSize -> 1089, p75FileSize -> 1089, p50FileSize -> 1089, numAddedBytes -> 1089)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
1,2025-07-12T13:51:28.000Z,5763104327656046,dshkr2002@gmail.com,UPDATE,"Map(predicate -> [""(id#10828L = 1)""])",null,null,0712-134530-qhpwm4jy-v2n,0,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3493, numDeletionVectorsUpdated -> 0, scanTimeMs -> 2106, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1030, rewriteTimeMs -> 1352)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
0,2025-07-12T13:49:06.000Z,5763104327656046,dshkr2002@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,null,0712-134530-qhpwm4jy-v2n,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 4, numOutputBytes -> 1076)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12


## zorder by

In [0]:
%sql

optimize delta.`/Volumes/delta/deltaschema/optimization/optimize/` zorder by (id)

path,metrics
dbfs:/Volumes/delta/deltaschema/optimization/optimize,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 1089), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1752328511178, 1752328511622, 8, 0, null, List(0, 0), null, 3, 3, 0, 0, null)"


In [0]:
%sql
describe history delta.`/Volumes/delta/deltaschema/optimization/optimize/`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-07-12T13:51:30.000Z,5763104327656046,dshkr2002@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,null,0712-134530-qhpwm4jy-v2n,1,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 2106, p25FileSize -> 1089, numDeletionVectorsRemoved -> 1, minFileSize -> 1089, numAddedFiles -> 1, maxFileSize -> 1089, p75FileSize -> 1089, p50FileSize -> 1089, numAddedBytes -> 1089)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
1,2025-07-12T13:51:28.000Z,5763104327656046,dshkr2002@gmail.com,UPDATE,"Map(predicate -> [""(id#10828L = 1)""])",null,null,0712-134530-qhpwm4jy-v2n,0,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3493, numDeletionVectorsUpdated -> 0, scanTimeMs -> 2106, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1030, rewriteTimeMs -> 1352)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
0,2025-07-12T13:49:06.000Z,5763104327656046,dshkr2002@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,null,0712-134530-qhpwm4jy-v2n,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 4, numOutputBytes -> 1076)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
